In [20]:
from sage.all import *
from collections import deque

In [26]:
TQDM_ON = True
if TQDM_ON:
    from tqdm import tqdm

In [30]:
def fast_pairwise_gcd_helper_recursive(nums:list):
    length = len(nums)
    if length<2:
        raise ValueError

    nums_sage = list(map(lambda x:Integer(x),nums))
    nums_square = list(map(lambda x:x**2,nums_sage))

    total_product = Integer(1)
    for num_s in nums_sage:
        total_product*=num_s
    
    sq_product_table = dict()
    def calculate_square(l,r):
        if l==r:
            sq_product_table[(l,l)]=nums_square[l]
            return
        m = (l+r)>>1
        calculate_square(l,m)
        l_res = sq_product_table[(l,m)]
        calculate_square(m+1,r)
        r_res = sq_product_table[(m+1,r)]
        sq_product_table[(l,r)] = l_res*r_res
    calculate_square(0,length-1)

    mod_ni_square = [None]*length
    def calculate_mode_ni_square(previous_mod_res,l,r):
        this_mod_res = previous_mod_res%sq_product_table[(l,r)]
        if l==r:
            mod_ni_square[l] = this_mod_res
            return
        m = (l+r)>>1
        calculate_mode_ni_square(this_mod_res,l,m)
        calculate_mode_ni_square(this_mod_res,m+1,r)
    m = (0+length-1)>>1
    calculate_mode_ni_square(total_product,0,m)
    calculate_mode_ni_square(total_product,m+1,length-1)

    return list(gcd(nums_sage[i],mod_ni_square[i]//nums_sage[i]) for i in range(length))





In [40]:
def fast_pairwise_gcd_helper(nums:list):
    length = len(nums)
    if length<2:
        raise ValueError
    length_padded = 0
    for i in range(32):
        if (1<<i)>=length:
            length_padded = 1<<i
            break
    nums+=[1]*(length_padded-length)
    origin_len,length = length,length_padded

    nums_sage = list(map(lambda x:Integer(x),nums))
    nums_square = list(map(lambda x:x**2,nums_sage))

    total_product = Integer(1)
    for num_s in (tqdm(nums_sage) if TQDM_ON else nums_sage):
        total_product*=num_s

    index_pair_list = list()
    bfs_queue = deque([(0,length-1)])
    while len(bfs_queue)>0:
        i_pair = bfs_queue.popleft()
        index_pair_list.append(i_pair)
        l,r=i_pair
        if l<r:
            m=(l+r)>>1
            bfs_queue.append((l,m))
            bfs_queue.append((m+1,r))
    # print(index_pair_list)
    
    sq_product_table = [0]*len(index_pair_list)
    irange = range(len(index_pair_list)-1,-1,-1)
    if TQDM_ON:
        irange = tqdm(irange)
    for i in irange:
        l,r = index_pair_list[i]
        if l==r:
            sq_product_table[i] = nums_square[l]
        else:
            sq_product_table[i] = sq_product_table[2*i+1]*sq_product_table[2*i+2]

    mod_ni_square_table = [0]*len(index_pair_list)
    mod_ni_square_table[0] = total_product
    irange = range(1,len(index_pair_list))
    if TQDM_ON:
        irange = tqdm(irange)
    for i in irange:
        mod_ni_square_table[i] = mod_ni_square_table[(i-1)>>1]%sq_product_table[i]
    
    mod_ni_square = [0]*length
    for i in range(len(index_pair_list)-1,-1,-1):
        l,r = index_pair_list[i]
        if l<r:
            break
        mod_ni_square[l] = mod_ni_square_table[i]
    
    return list(gcd(nums_sage[i],mod_ni_square[i]//nums_sage[i]) for i in range(length))[:origin_len]


In [33]:
fast_pairwise_gcd_helper([6,9,11,17,13])

100%|██████████| 14/14 [00:00<00:00, 170698.42it/s]


[3, 3, 1, 1, 1]

In [34]:
l = [17,13,101*2,101*3,61,79]
print(fast_pairwise_gcd_helper_recursive(l))
print(fast_pairwise_gcd_helper(l))

[1, 1, 101, 101, 1, 1]


100%|██████████| 14/14 [00:00<00:00, 109145.46it/s]

[1, 1, 101, 101, 1, 1]


In [37]:
with open("./moduli.sorted") as fin:
    lines = fin.readlines()
    ns = list(map(lambda line:int(line.strip(),16),lines))

In [43]:
nums = ns

length = len(nums)
if length<2:
    raise ValueError
length_padded = 0
for i in range(32):
    if (1<<i)>=length:
        length_padded = 1<<i
        break
nums+=[1]*(length_padded-length)
origin_len,length = length,length_padded

nums_sage = list(map(lambda x:Integer(x),nums))
nums_square = list(map(lambda x:x**2,nums_sage))

# total_product = Integer(1)
# for num_s in (tqdm(nums_sage) if TQDM_ON else nums_sage):
#     total_product*=num_s

100%|██████████| 131072/131072 [18:37<00:00, 117.28it/s]


In [45]:
# len(str(total_product))

30738564

In [56]:
index_pair_list = list()
bfs_queue = deque([(0,length-1)])
while len(bfs_queue)>0:
    i_pair = bfs_queue.popleft()
    index_pair_list.append(i_pair)
    l,r=i_pair
    if l<r:
        m=(l+r)>>1
        bfs_queue.append((l,m))
        bfs_queue.append((m+1,r))
# print(index_pair_list)
    
sq_product_table = [0]*len(index_pair_list)
irange = range(len(index_pair_list)-1,-1,-1)
if TQDM_ON:
    irange = tqdm(irange)
for i in irange:
    l,r = index_pair_list[i]
    if l==r:
        sq_product_table[i] = nums_square[l]
    else:
        sq_product_table[i] = sq_product_table[2*i+1]*sq_product_table[2*i+2]


100%|██████████| 262143/262143 [00:06<00:00, 40778.43it/s]  


In [57]:
product_table = [0]*len(index_pair_list)
irange = range(len(index_pair_list)-1,-1,-1)
if TQDM_ON:
    irange = tqdm(irange)
for i in irange:
    l,r = index_pair_list[i]
    if l==r:
        product_table[i] = nums_sage[l]
    else:
        product_table[i] = product_table[2*i+1]*product_table[2*i+2]

100%|██████████| 262143/262143 [00:02<00:00, 87525.06it/s]  


In [59]:
total_product=product_table[0]

In [48]:
len(str(sq_product_table[0]))

61477127

In [47]:
mod_ni_square_table = [0]*len(index_pair_list)
mod_ni_square_table[0] = total_product
irange = range(1,len(index_pair_list))
if TQDM_ON:
    irange = tqdm(irange)
for i in irange:
    mod_ni_square_table[i] = mod_ni_square_table[(i-1)>>1]%sq_product_table[i]
    
mod_ni_square = [0]*length
for i in range(len(index_pair_list)-1,-1,-1):
    l,r = index_pair_list[i]
    if l<r:
        break
    mod_ni_square[l] = mod_ni_square_table[i]
    
gcd_helper_res= list(gcd(nums_sage[i],mod_ni_square[i]//nums_sage[i]) for i in range(length))[:origin_len]

100%|██████████| 262142/262142 [00:27<00:00, 9627.67it/s]  


In [50]:
have_gcd_lines_i = list()

for i,r in enumerate(gcd_helper_res):
    if r>1:
        have_gcd_lines_i.append(i)

In [51]:
have_gcd_lines_i

[71679, 81922]

In [52]:
a,b = have_gcd_lines_i

In [63]:
n_a,n_b = int(lines[a],16),int(lines[b],16)

In [66]:
p = int(gcd(n_a,n_b))

In [68]:
q_a = n_a//p
q_b = n_b//p

int